# Association rule mining

Goal

### Algorithm
![algorithm](https://ndownloader.figshare.com/files/7635700?private_link=3c473609741895a5cc2c)

### Dataset
Each line of this file is some customer's shopping basket. The items that the customer bought are stored as a comma-separated list of values.

In [3]:
def download(file, local_dir="", url_base=None, checksum=None):
    import os, requests, hashlib, io
    local_file = "{}{}".format(local_dir, file)
    if not os.path.exists(local_file):
        if url_base is None:
            url_base = "https://cse6040.gatech.edu/datasets/"
        url = "{}{}".format(url_base, file)
        print("Downloading: {} ...".format(url))
        r = requests.get(url)
        with open(local_file, 'wb') as f:
            f.write(r.content)            
    if checksum is not None:
        with io.open(local_file, 'rb') as f:
            body = f.read()
            body_checksum = hashlib.md5(body).hexdigest()
            assert body_checksum == checksum, \
                "Downloaded file '{}' has incorrect checksum: '{}' instead of '{}'".format(local_file,
                                                                                           body_checksum,
                                                                                           checksum)
    print("'{}' is ready!".format(file))
    
DATA_PATH = ""
datasets = {'groceries.csv': '0a3d21c692be5c8ce55c93e59543dcbe'}

for filename, checksum in datasets.items():
    download(filename, local_dir=DATA_PATH, checksum=checksum)

with open('{}{}'.format(DATA_PATH, 'groceries.csv')) as fp:
    groceries_file = fp.read()
print (groceries_file[0:250] + "...\n... (etc.) ...") # Prints the first 250 characters only
print("\n(All data appears to be ready.)")

'groceries.csv' is ready!
citrus fruit,semi-finished bread,margarine,ready soups
tropical fruit,yogurt,coffee
whole milk
pip fruit,yogurt,cream cheese ,meat spreads
other vegetables,whole milk,condensed milk,long life bakery product
whole milk,butter,yogurt,rice,abrasive clea...
... (etc.) ...

(All data appears to be ready.)


### Preprocess data

In [14]:
def get_processed_data(text):
    reciepts = groceries_file.split('\n')
    itemsets = [set(r.split(',')) for r in reciepts]
    return itemsets

reciepts = get_processed_data(groceries_file)

### Create dictionaries

In [23]:
from collections import defaultdict
from itertools import combinations

pair_counts = defaultdict(int)
item_counts = defaultdict(int)

for r in reciepts:
    for (a,b) in combinations(r, 2):
        pair_counts[(a,b)] += 1
        pair_counts[(b,a)] += 1
    for c in r:
        item_counts[c] += 1

### Get associations

In [32]:
THREHSOLD = 0.5
MIN_COUNT = 10

rules = {} # (item_a, item_b) -> conf (item_a => item_b)

for (a,b), val in pair_counts.items():
    if item_counts[a] >= MIN_COUNT:
        conf = val/item_counts[a]
        if conf >= THREHSOLD:
            rules[(a,b)] = conf

In [34]:
# just some pretty printing
def gen_rule_str(a, b, val=None, val_fmt='{:.3f}', sep=" = "):
    text = "{} => {}".format(a, b)
    if val:
        text = "conf(" + text + ")"
        text += sep + val_fmt.format(val)
    return text

def print_rules(rules):
    if type(rules) is dict or type(rules) is defaultdict:
        from operator import itemgetter
        ordered_rules = sorted(rules.items(), key=itemgetter(1), reverse=True)
    else: # Assume rules is iterable
        ordered_rules = [((a, b), None) for a, b in rules]
    for (a, b), conf_ab in ordered_rules:
        print(gen_rule_str(a, b, conf_ab))

print_rules(rules)

conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
conf(cocoa drinks => whole milk) = 0.591
conf(pudding powder => whole milk) = 0.565
conf(jam => whole milk) = 0.547
conf(cream => other vegetables) = 0.538
conf(cream => sausage) = 0.538
conf(baking powder => whole milk) = 0.523
conf(tidbits => rolls/buns) = 0.522
conf(rice => other vegetables) = 0.520
conf(cooking chocolate => whole milk) = 0.520
conf(specialty cheese => other vegetables) = 0.500
conf(rubbing alcohol => citrus fruit) = 0.500
conf(rubbing alcohol => butter) = 0.500
conf(ready soups => rolls/buns) = 0.500
conf(frozen fruits => whipped/sour cream) = 0.500
